In [ ]:
import tensorflow as tf
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load intents data from JSON file
with open('../data/intents.json', 'r') as file:
    intents = json.load(file)

# Extract patterns and intents labels from the data
patterns = []
labels = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        labels.append(intent['tag'])

# Tokenize patterns
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(patterns)
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Define vocabulary size and embedding dimension
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16

# Build a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(set(encoded_labels)), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(padded_sequences, encoded_labels, epochs=50)

# Function to predict intent from user input
def predict_intent(user_input):
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, padding='post', maxlen=padded_sequences.shape[1])
    prediction = model.predict(padded_sequence)
    predicted_label = np.argmax(prediction, axis=1)[0]
    predicted_intent = label_encoder.inverse_transform([predicted_label])[0]
    return predicted_intent

# Test the chatbot with user inputs
user_inputs = ["hello", "bye", "thanks", "how are you?"]

for user_input in user_inputs:
    intent = predict_intent(user_input)
    print(f"User: {user_input}")
    print(f"Bot: Intent - {intent}")
    print()
